In [1]:
import MySQLdb
from Tkinter import *
import ttk

class netflixInterface:
    def __init__(self, master):
        
        #Set Master
        self.master = master
        
        #Set Title 
        master.title("CSE 305 - Netflix Database")
        
        #Set Window Width*Heigth
        master.geometry("500x500")
        
        #Set Database Connection
        self.connection = MySQLdb.connect("localhost", "testuser", "test1234", "netflix")
        self.cur = self.connection.cursor()
        
        #Sets up GUI 
        self._setup_widget(master)
        
    def _setup_widget(self, master):
        #Display Database Name
        self.label = Label(master, text = "Netflix Database")
        self.label.place(x = 200, y = 0)
        
        #Select Data to Query Label
        self._select_query_type = Label(master, text = "Select Data to Query:")
        self._select_query_type.place(x=10, y=50)
        
        #CheckButton Container for Users and Shows
        self._checkbuttonFrame1 = Frame(master)
        self._checkbuttonFrame1.place(x = 10, y = 80)
        
        #CheckButton for Users
        self.userBox = IntVar()
        self.user_checkbutton = Checkbutton(self._checkbuttonFrame1, text="Users", variable=self.userBox, command=self.displayOptionFrame)
        self.user_checkbutton.grid(row=0, column=0)
        
        #Checkbutton for Shows
        self.showBox = IntVar()
        self.shows_checkbutton = Checkbutton(self._checkbuttonFrame1, text="Shows",variable=self.showBox, command=self.displayOptionFrame)
        self.shows_checkbutton.grid(row=0, column=1)
        
        #QueryFrame
        self._queryOptionFrame = Frame(self.master)
        self._queryOptionFrame.place(x = 0, y = 110)
        
        #Display Frame
        self._data_frame = Frame(self.master)
        self._data_frame.pack(side = RIGHT)
        
    def displayOptionFrame(self):
    
        #Remove Query Frame's Children
        for item in self._queryOptionFrame.winfo_children():
            item.grid_forget()
        
        #Remove Data Frame's Children
        for item in self._data_frame.winfo_children():
            item.grid_forget()
            
        if(self.userBox.get()):
        
            #Disable show checkbutton
            self.shows_checkbutton.config(state = DISABLED)
            
            #Select User Label
            self.select_user_label = Label(self._queryOptionFrame, text = "Select a User: ")
            self.select_user_label.grid(row = 0, column = 0)

            #User DropDown Menu
            self.menuChoice = StringVar(self._queryOptionFrame)
        
            self.userChoices = ["User 1", "User 2", "User 3", "User 4", "User 5", 
                            "User 6", "User 7", "User 8", "User 9", "User 10"]
        
            self.menuChoice.set("User 1")
                
            self._user_menu = OptionMenu(self._queryOptionFrame, self.menuChoice, *self.userChoices)
            self._user_menu.grid(row = 1, column = 0)
            
            #Recommended Shows Button 
            self.recommendShow = Button(self._queryOptionFrame, text="Recommended Shows", command = self.recommendedShowButton)
            self.recommendShow.grid(row = 1, column = 2)
    
            #Favorite Shows Button 
            self.favoriteShow = Button(self._queryOptionFrame, text="Favorite Shows", command = self.favoriteShowButton)
            self.favoriteShow.grid(row = 2, column = 2)
            
        if(self.showBox.get()):
            
            #Disable user checkbutton
            self.user_checkbutton.config(state = DISABLED)
            
            #Title Widget
            self._title_prompt = Label(self._queryOptionFrame, text = "Enter the title of the show to search for : ", justify = "left")
            self._title_prompt.grid(row = 0, column = 0, sticky=W)
        
            #Title Widget
            self._title = Label(self._queryOptionFrame, text = "Title")
            self._title.grid(row = 1, column = 0, sticky=W)
        
            #Textbox for Title - Reads User's Input
            self._title_txtbox = Entry(self._queryOptionFrame, width = 20)
            self._title_txtbox.grid(row = 1, column = 1, sticky=W)
        
            #Select Type of Show Label
            self._select_query_type = Label(self._queryOptionFrame, text = "Select Type of show:")
            self._select_query_type.grid(row = 3, column = 0, sticky=W)
        
            #CheckButton for Movies
            self.movieBox = IntVar()
            self.movies_checkbutton = Checkbutton(self._queryOptionFrame, text="Movies", variable=self.movieBox)
            self.movies_checkbutton.grid(row=5, column=0, sticky=W)

            #Checkbutton for TV Shows
            self.tvshowBox = IntVar()
            self.tvshows_checkbutton = Checkbutton(self._queryOptionFrame, text="TV Shows",variable=self.tvshowBox)
            self.tvshows_checkbutton.grid(row=5, column=1, sticky=W)

            #Genre Widget
            self._genre = Label(self._queryOptionFrame, text = "Specify Genre: ")
            self._genre.grid(row = 7, column = 0, sticky=W)

            #Textbox for Genre - Reads User's Input
            self._genre_txtbox = Entry(self._queryOptionFrame, width = 10)
            self._genre_txtbox.grid(row = 7, column = 1, sticky=W)
        
            #Search Button 
            self.search_button = Button(self._queryOptionFrame, text="Search", command=self.searchButton)
            self.search_button.grid(row = 9, column = 0, sticky=W + E, padx=60)
            
            
        #Enables disabled checkbutton 
        if(not self.showBox.get()):
            self.user_checkbutton.config(state=NORMAL)   
            
        if(not self.userBox.get()):
            self.shows_checkbutton.config(state=NORMAL)
            
            
    def recommendedShowButton(self):
        args = []
        
        getUserNumber = "%"
        getUserNumber += self.menuChoice.get()[5:7]
        getUserNumber += "%"
        
        args.append(getUserNumber)
       
        query = "SELECT shows.name, shows.description, shows.genre, shows.releaseYear FROM user, shows, recommendedshows WHERE username LIKE %s AND user.id = recommendedshows.userId AND shows.id = recommendedshows.showid"
        
        
        self.cur.execute(query, args)    
        self.displayData(self.cur.fetchall())
        
    def favoriteShowButton(self):
        args = []
        
        getUserNumber = "%"
        getUserNumber += self.menuChoice.get()[5:7]
        getUserNumber += "%"
        
        args.append(getUserNumber)
       
        query = "SELECT shows.name, shows.description, shows.genre, shows.releaseYear FROM user, shows, favoriteshows WHERE username LIKE %s AND user.id = favoriteshows.userId AND shows.id = favoriteshows.showid"
        
        
        self.cur.execute(query, args)    
        self.displayData(self.cur.fetchall())
        
        
            
    def searchButton(self):
        
        if not self.movieBox.get() and not self.tvshowBox.get():
            self.displayData([[]])
            return
        
        args = []
        query = "SELECT name, description, genre, releaseYear FROM shows WHERE name LIKE %s AND genre LIKE %s"
        
        title = self._title_txtbox.get()
        title += "%"
        args.append(title)
        
        genre = self._genre_txtbox.get()
        genre += "%"
        args.append(genre)
        
        
        if self.movieBox.get() and self.tvshowBox.get():
            query = "SELECT name, description, genre, releaseYear FROM shows WHERE name LIKE %s AND genre LIKE %s"
            
        else:
            if self.movieBox.get():
                 query = "SELECT shows.name, shows.description, shows.genre, shows.releaseYear, movies.rating FROM shows, movies WHERE name LIKE %s AND genre LIKE %s AND shows.id = movies.id"
            else:
                query = "SELECT shows.name, shows.description, shows.genre, shows.releaseYear, tvshows.rating FROM shows, tvshows WHERE name LIKE %s AND genre LIKE %s AND shows.id = tvshows.id"
            
        self.cur.execute(query, args)
        self.displayData(self.cur.fetchall())
                
    def displayData(self, array):
        
        def configFunction(event):
            canvas.configure(scrollregion=canvas.bbox("all"),width=1000,height=1000)
        
        #Remove Frame's Children
        for item in self._data_frame.winfo_children():
            item.destroy()
            
        vscrollbar = Scrollbar(self._data_frame, orient=VERTICAL)
        vscrollbar.pack(fill=Y, side=RIGHT, expand=FALSE)
        canvas = Canvas(self._data_frame, bd=0, highlightthickness=0,
                        yscrollcommand=vscrollbar.set)
        canvas.pack(side=LEFT, fill=BOTH, expand=TRUE)
        vscrollbar.config(command=canvas.yview)

        # reset the view
        canvas.xview_moveto(0)
        canvas.yview_moveto(0)

        # create a frame inside the canvas which will be scrolled with it
        interior = Frame(canvas)
        
        #Display Data from Database onto frame
        for index, data in enumerate(array):
            ttk.Separator(interior, orient = HORIZONTAL).grid(row = 2 * index, columnspan = len(data), sticky = "ew")
            for i in range(len(data)):
                Label(interior, text = data[i], justify = "left", wraplength = 400, padx = 3, pady = 3).grid(row = 2 * index + 1, column = i)
        
        
        interior_id = canvas.create_window(0, 0, window=interior,
                                           anchor=NW)
        interior.bind("<Configure>", configFunction)
        
        
      
    
    
root = Tk()
gui = netflixInterface(root)
root.mainloop()
